## <1-3 학습 파이프라인 소개>  

이번 절에서는 모델 학습의 전체 파이프라인을 소개합니다. 이 파이프라인은 이 책에서 소개하는 5가지 과제(문서 분류, 개체명 인식, 질의응답, 문서 검색, 문장 생성)에 상관없이 공통으로 적용됩니다.  

이 책에서 진행하는 모든 실습은 ***ratsnlp*** 라는 오픈소스 파이썬 패키지를 사용합니다. 코랩 환경에서 책의 모든 실습을 진행할 수 있도록 필자가 직접 개발했습니다.

### 각종 설정값 정하기  
---
모델을 만들려면 가장 먼저 각종 설정값을 정해야 합니다. 어떤 프리트레인 모델을 사용할지, 학습에 사용할 데이터는 무엇인지, 학습 결과는 어디에 저장할지 등이 바로 그것입니다. 이 설정값들은 본격적인 학습에 앞서 미리 선언해 둡니다. 다음 코드는 4장에서 살펴볼 문서 분류를 위한 각종 설정값을 선언한 예입니다.  

***하이퍼 파라미터(hyperparameter)***  역시 미리 정해둬야 하는 중요한 정보입니다. 하이퍼파라미터란 모델 구조와 학습 등에 직접 관계된 설정값을 가리킵니다. 예를 들어 ***러닝 레이트(learning rate)***, ***배치 크기(batch size)*** 등이 있습니다.

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 670 kB/s 
     |████████████████████████████████| 57 kB 5.9 MB/s 
     |████████████████████████████████| 2.8 MB 41.8 MB/s 
     |████████████████████████████████| 806 kB 45.3 MB/s 
     |████████████████████████████████| 397 kB 25.8 MB/s 
     |████████████████████████████████| 829 kB 54.0 MB/s 
     |████████████████████████████████| 134 kB 57.0 MB/s 
     |████████████████████████████████| 636 kB 68.2 MB/s 
     |████████████████████████████████| 895 kB 62.8 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.7 MB/s 
     |████████████████████████████████| 96 kB 7.5 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 271 kB 75.2 MB/s 
     |████████████████████████████████| 144 kB 74.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [2]:
#설정값 선언
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir="/root/Korpora",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    learning_rate=5e-5,
    batch_size=32
)

### 데이터 내려받기
---
이 책에서는 프리트레인을 마친 모델을 다운스트림 데이터로 파인튜닝하는 실습을 진행합니다. 파인튜닝을 하려면 다운스트림 데이터를 미리 내려받아 둬야 합니다. 박은정 님이 공개한 네이버 영화 리뷰 말뭉치인 ***NSMC(Naver Sentiment Movie Corpus)*** 가 대표적입니다.   
다음 코드는 downstream_corpus_name에 해당하는 말뭉치를 내려받아 downstream_corpus_root_dir 아래에 저장합니다. 즉, 위 코드에서 설정한 args에 따라 nsmc를 /root/Korpora 디렉터리에 저장합니다.

In [3]:
#데이터 다운로드
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 106MB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:00, 42.9MB/s]                            


코드에서 확인할 수 있는 것처럼 이 책에서는 다운로드 툴킷으로 ***ratsnlp*** 뿐 아니라 ***코포라(Korpora)*** 라는 오픈소스 파이썬 패키지를 사용합니다. 이 패키지는 다양한 한국어 말뭉치를 쉽게 내려받고 전처리할 수 있도록 도와줍니다.

### 프리트레인을 마친 모델 준비하기
---  
대규모 말뭉치를 활용한 프리트레인에는 많은 리소스가 필요합니다. 다행히 최근 많은 기업과 개인이 프리트레인을 마친 모델을 자유롭게 사용할 수 있도록 공개하고 있어서 그 혜택을 볼 수 있습니다.  

특히 미국 자연어 처리 기업 허깅페이스(huggingface)에서 만든 ***트랜스포머(transformers)*** 라는 오픈소스 파이썬 패키지에 주목해야 합니다. 이 책에서는 BERT, GPT 같은 트랜스포머 계열 모델로 실습을 진행하는데, 이 패키지를 쓰면 단 몇 줄만으로 모델을 사용할 수 있습니다.  

다음 코드는 이준범 님이 허깅페이스 모델 허브에 등록한 kcbert-base 모델을 준비하는 코드입니다. 앞서 보인 코드에서 args.pretrained_model_name에 beomi/kcbert-base라고 선언해 뒀으므로 위 코드와 곧이어 아래 나올 코드를 차례로 실행하면 kcbert-base 모델을 쓸 수 있는 상태가 됩니다.

In [4]:
#kcbert-base 모델 준비
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=2,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

이 코드를 실행하면 kcbert-base가 로컬 저장소에 없으면 자동으로 내려받고, 있으면 캐시 디렉터리에서 읽어옵니다.

### 토크나이저 준비하기
---
자연어 처리 모델의 입력은 대개 ***토큰(token)***입니다. 여기서 토큰이란 ***문장(sentence)*** 보다 작은 단위입니다. 한 문장은 여러 개의 토큰으로 구성됩니다. 토큰 분리 기준은 그때그때 다를 수 있습니다. 문장을 띄어쓰기만으로 나눌 수도 있고, 의미의 최소 단위인 ***형태소(morpheme)*** 단위로 나눌수도 있습니다.

문장을 ***토큰 시퀀스(token sequence)*** 로 분석하는 과정을 ***토큰화(tokenization)*** , 토큰화를 수행하는 프로그램은 ***토크나이저(tokenizer)*** 라고 합니다. 이 책에서는 ***BPE(Byte Pair Encoding)*** 나 ***워드피스(wordpiece)*** 알고리즘을 채택한 토크나이저를 실습에 활용합니다.  

다음 코드는 ***kcbert-base*** 모델이 사용하는 토크나이저를 준비하는 코드입니다. 이 역시 토크나이저 관련 파일이 로컬 저장소에 없으면 자동으로 내려받고, 있으면 캐시에서 읽어옵니다.

In [5]:
#kcbert-base 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

### 데이터 로더 준비하기
---
***파이토치(Pytorch)*** 는 딥러닝 모델의 학습을 지원하는 파이썬 라이브러리입니다. 파이토치에는 ***데이터 로더(DataLoader)*** 라는 게 포함돼 있습니다. 파이토치로 딥러닝 모델을 만들려면 이 데이터 로더를 반드시 정의해야 합니다.  

데이터 로더는 데이터를 ***배치(batch)*** 단위로 모델에 밀어 넣어주는 역할을 합니다. 전체 데이터 가운데 일부 인스턴스를 뽑아(sample) 배치를 구성합니다. ***데이터셋(dataset)***은 데이터 로더의 구성 요소 가운데 하나입니다. 데이터셋은 여러 인스턴스(문서+레이블)를 보유하고 있습니다. 다음 그림에서는 편의를 위해 인스턴스가 10개인 데이터셋을 상정했지만 대개 인스턴스 개수는 이보다 훨씬 많습니다.  

데이터 로더가 배치를 만들 때 인스턴스를 뽑는 방식은 파이토치 사용자가 자유롭게 정할 수 있습니다.  
배치는 그 모양이 고정적이어야 할 때가 많습니다. 다시 말해 동일한 배치에 있는 문장들의 토큰(input_ids) 개수가 같아야 합니다.  
예를 들어 이번에 만들 배치가 데이터셋의 0번, 3번, 6번 인스턴스이고 각각의 토큰 개수가 5, 3, 4개라고 가정해 보겠습니다. 제일 긴 길이로 맞춘다면 0번 인스턴스의 길이(5개)에 따라 3번과 6번 인스턴스의 길이를 늘여야 합니다.

이처럼 배치의 모양 등을 정비해 모델의 최종 입력으로 만들어 주는 과정을 ***컬레이트(collate)*** 라고 합니다. 컬레이트 과정에는 파이썬 ***리스트(list)*** 에서 파이토치 ***텐서(tensor)*** 로의 변환 등 자료형 변환도 포함됩니다. 컬레이트 수행 방식 역시 파이토치 사용자가 자유롭게 구성할 수 있습니다.  

다음은 문서 분류를 위한 데이터 로더를 준비하는 예시입니다. 

In [6]:
!pip install nlpbook

     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 189 kB 69.2 MB/s 
     |████████████████████████████████| 55 kB 5.4 MB/s 
     |████████████████████████████████| 198 kB 63.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2bad8a75c1d9535a02265dfba2712ddf2ad00d56282c746c9f336d6f596f4b01
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [7]:
#문서 분류 데이터 로더 선언
from torch.utils.data import DataLoader, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

NameError: ignored